![EC JRC](https://github.com/ec-jrc/cbm/blob/main/docs/img/eu_science_hub.png?raw=true)

# JRC RESTful service example

This Jupyter Notebook provides examples on how to view data provided from the JRC RESTful API with the use of cbm python package


    This file is part of CbM (https://github.com/ec-jrc/cbm).
    Copyright : 2021 European Commission, Joint Research Centre
    License   : 3-Clause BSD


See the documentation for more details at: https://jrc-cbm.readthedocs.io/en/latest/api_overview.html

Use the python library ```requests``` to manually configure and make requests, or the ```cbm``` library to easily get data from the RESTful server.


---
---
## With python requests

In [ ]:
# RESTful API Requests example with the use of requests library
# The '/query/info' requests will return information about the server and the available data.
import json
import requests

api_url = 'https://cap.users.creodias.eu'
api_user = 'USERNAME'
api_pass = 'PASSWORD'

requrl = "{}/query/info"
response = requests.get(requrl.format(api_url),
                        auth=(api_user, api_pass))

print(json.dumps(json.loads(response.content), indent=1))

---
---

## The 'cbm' python library
A tool to easily explore the data and check individual parcels or small set of parcels.

See the documentation for more details at:https://jrc-cbm.readthedocs.io/en/latest/cbm_overview.html

In [ ]:
# Install or upgrade cbm (Restart of the Notebook kerner may be required to take effect)
!pip install cbm --upgrade

In [ ]:
import cbm
print(cbm.__version__)

**Planned future updates**
- 0.0.13-14: Add multi parcel IDs selection for comparison of S1 plots.
- 0.0.14-15: Improved functionalities and basic views.
- 0.0.15-17: Fix direct database and object storage access functions.
- 0.0.18-21: Fix and update signatures extraction functions.
- ~0.0.22-23: Integrate calendar view ([calendar_view_gui](https://github.com/ec-jrc/cbm/tree/main/scripts/calendar_view_gui)).
- '>0.0.24: Add marker processing-detection functions ([signal_marker_processing](https://github.com/ec-jrc/cbm/tree/main/scripts/signal_marker_processing)).

In [ ]:
cbm.set_api_account(api_url, api_user, api_pass) # set your credentials

In [ ]:
# Select parcel
aoi='MS'        # Area of interest (str)
year='2020'     # Year of the parcels dataset (int)
pid='123456'    # Parcel id (int)
ptype=''        # Parcel type (Optional), use only in case there are separate parcel datasets for
                #   the same year dedicated to different type of analyses or different regions),
                #   to see the available parcel types go to: https://cap.users.creodias.eu/query/info

cbm.show.parcel_info.by_pid(aoi, year, pid, ptype)

In [ ]:
# Show parcel backround images, for some member states or regions, orthophotos are provided as well.
# To see the available orthophotos go to: https://cap.users.creodias.eu/query/info


cbm.show.background.by_pid(
    aoi=aoi,
    year=year,
    pid=pid,
    chipsize=256,  # Pixels
    extend=800.0,  # Meters
    tms=['google', 'bing', 'ags', 'osm'],
    ptype=ptype
)

### Sentinel 2 images

In [ ]:
# Show list of Sentinel images in a grid
# This takes some time to download the images, do not use this for large amount of parcels or long time period.
chipsize = 512
start_date = '2020-06-01'
end_date = '2020-06-10'
imgtype = 'true color' # imgtype options:
#                            - Multi band, e.g.:['B03', 'B04', 'B08']
#                            - Single band, e.g: 'B08'
#                            - Or 'true color' and 'ndvi'

# Other default options:
# clean_history = False # Delete the previous chip_images folder
# debug = False         # Show text output of the procedures
# columns = 4           # Number of columns of the grid
# show_parcel = True    # Show or not the parcel polygon
# percentile = [2, 98]  # Color levels percentage
# cmap = 'RdYlGn_r'     # Color map only for NDVI

cbm.show.chip_images.grid(aoi, year, pid, start_date, end_date, imgtype, chipsize, ptype=ptype)

In [ ]:
# Show NDVI images in a grid
imgtype = 'ndvi'
cbm.show.chip_images.grid(aoi, year, pid, start_date, end_date, imgtype, chipsize, ptype=ptype)

### Sentinel 1/2 signatures time series

In [ ]:
# Plot Sentinel 2 time series for selected bands
cbm.show.time_series.s2(aoi, year, pid, bands=['B02', 'B03', 'B04', 'B08'], scl='3_8_9_10_11', ptype=ptype)

In [ ]:
# Plot NDVI profile for the selected parcel
cbm.show.time_series.ndvi(aoi, year, pid, ptype=ptype)

In [ ]:
# Find parcels with the same crop type within a given distance
parcel_peers = cbm.get.parcels_list.peers(aoi, year, pid, distance=1000, maxPeers=5, ptype=ptype)
parcel_peers

In [ ]:
# Plot profiles of the parcels peers
cbm.show.time_series.ndvi(aoi, year, parcel_peers['pids'], ptype=ptype)

In [ ]:
# Compare the profiles of the selected parcel with its parcel peers
# Find parcels with the same crop type within a given distance

parcel_peers2 = cbm.get.parcels_list.peers(aoi, year, pid, distance=2000, maxPeers=10, ptype=ptype)

cbm.show.time_series.ndvi(aoi, year, [pid]+parcel_peers2['pids'], ptype=ptype, max_line_plots=1)

In [ ]:
# Plot the Sentinel-1 Backscattering coefficient.
cbm.show.time_series.s1(aoi, year, pid, 'bs', ptype)

In [ ]:
# Plot the Sentinel-1 6 day coherence.
cbm.show.time_series.s1(aoi, year, pid, 'c6', ptype)

---
# Jupyter notebooks interactive widgets for CbM **'ipycbm'**

The ipycbm subpackage of cbm python library provides an easy way to get and view data with the use of python widgets for notebooks 'ipywidgets' also known as jupyter-widgets, the interactive HTML widgets for Jupyter notebooks and the IPython kernel.
For more details see see the ["CbB overview"](https://jrc-cbm.readthedocs.io/en/latest/cbm_overview.html) chapter of the documentation.


In [ ]:
from cbm import ipycbm # Import the python library ipycbm
ipycbm.view()          # Display the data with easy selectable predefined view options.